In [10]:
%run ./work/lib.py

In [11]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType, DecimalType, TimestampType, DataType, IntegerType
from pyspark.sql.functions import col, lit, trim, substring, concat, udf, upper, initcap
from datetime import datetime
import os

In [12]:
conf = SparkConf() \
    .setAppName("projeto_pbi") \
    .setSparkHome('./spark/home')

LAKE_HOME = os.getenv("LAKE_HOME", "/spark/home")

sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

22/12/17 22:52:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [13]:
TABLE_NAME = 'fact_order_item'

In [14]:
df = spark.read.csv('{}/dataset/e-commerce/01_extract/order_items_dataset.csv'.format(LAKE_HOME), header=True)

df_item = \
    df.select(
        col('order_id').cast(StringType()).alias('ID_ORDER'),
        col('order_item_id').cast(StringType()).alias('NUM_ORDER_ITEM'),
        col('product_id').cast(StringType()).alias('ID_PRODUCT'),
        col('seller_id').cast(StringType()).alias('ID_SELLER'),
        col('shipping_limit_date').cast(TimestampType()).alias('DAT_SHIPPING_LIMIT'),
        col('price').cast(DecimalType(15,2)).alias('VAL_PRICE'),
        col('freight_value').cast(DecimalType(15,2)).alias('VAL_FREIGHT'),
    )

order_sk = map_sk(spark=spark, table_name='fact_order')
product_sk = map_sk(spark=spark, table_name='dim_product')
seller_sk = map_sk(spark=spark, table_name='dim_seller')

In [15]:
fact_order_item = \
    df_item \
        .withColumn('SK_ORDER', locate_sk(order_sk)(col('ID_ORDER'))) \
        .withColumn('SK_PRODUCT', locate_sk(product_sk)(col('ID_PRODUCT'))) \
        .withColumn('SK_SELLER', locate_sk(seller_sk)(col('ID_SELLER')))

In [16]:
fact_order_item = \
    fact_order_item.select(
            col('SK_ORDER'),
            col('SK_PRODUCT'),
            col('SK_SELLER'),
            col('NUM_ORDER_ITEM'),
            col('DAT_SHIPPING_LIMIT'),
            col('VAL_PRICE'),
            col('VAL_FREIGHT'),  
        )

fact_order_item.write.mode('overwrite').csv('{}/dataset/e-commerce/04_fact/{}.csv'.format(LAKE_HOME, TABLE_NAME), header=True)

In [17]:
spark.stop()